In [128]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
import Heston_pricer as hp  # Ton module perso

In [ ]:
#étape1 : récupérer les données de marchés (que des calls)
df = pd.read_excel(r"C:\Users\malu\Documents\Perso\Memoire\Vdef\BDD.xlsx") # Change this path!!!!!!
df = df[df["Type"].str.lower() == "call"] # On ne garde que les calls 
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot
0,Amazon,call,240,2025-07-18,0.070,0.08,219
1,Amazon,call,230,2025-07-18,0.270,0.28,219
2,Amazon,call,220,2025-07-18,0.850,0.88,219
3,Amazon,call,250,2025-07-18,0.001,0.05,219
4,Amazon,call,255,2025-08-15,0.210,0.22,219
5,Amazon,call,280,2025-09-19,0.110,0.12,219
6,Amazon,call,245,2025-08-15,0.410,0.42,219
7,Amazon,call,260,2025-09-19,0.330,0.34,219
8,Amazon,call,210,2025-07-18,1.960,1.99,219
9,Amazon,call,235,2025-08-15,0.800,0.80,219


In [130]:
#Ajout d'une colonne mid qui nous sert de prix
df ['Mid']=(df['Bid']+df['Ask'])/2
df.head()

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid
0,Amazon,call,240,2025-07-18,0.070,0.08,219,0.0750
1,Amazon,call,230,2025-07-18,0.270,0.28,219,0.2750
2,Amazon,call,220,2025-07-18,0.850,0.88,219,0.8650
3,Amazon,call,250,2025-07-18,0.001,0.05,219,0.0255
4,Amazon,call,255,2025-08-15,0.210,0.22,219,0.2150


In [ ]:
#étape2 : calculer la vol implicite pour chacune de ces options de marchés 

import Implied_volatility as iv

reference_date = pd.Timestamp('2025-07-01') # date à laquelle les données ont été extraites (sur le site de la société générale)

df['T'] = (df['Maturité'] - reference_date).dt.days / 365.0

r=0.02 # risk free

# Calcul de la volatilité implicite des options sur le marché
df['VI_market'] = df.apply(
    lambda row: iv.call_implied_volatility_mc(
        row['Mid'], 
        row['Spot'], 
        row['Prix d\'exercice'], 
        row['T'], 
        r
    ), 
    axis=1
)
df.dropna(inplace=True) # Supprime les lignes avec NaN
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market
0,Amazon,call,240,2025-07-18,0.070,0.08,219,0.0750,0.046575,0.204805
1,Amazon,call,230,2025-07-18,0.270,0.28,219,0.2750,0.046575,0.158086
2,Amazon,call,220,2025-07-18,0.850,0.88,219,0.8650,0.046575,0.064673
3,Amazon,call,250,2025-07-18,0.001,0.05,219,0.0255,0.046575,0.244612
4,Amazon,call,255,2025-08-15,0.210,0.22,219,0.2150,0.123288,0.226159
5,Amazon,call,280,2025-09-19,0.110,0.12,219,0.1150,0.219178,0.229892
6,Amazon,call,245,2025-08-15,0.410,0.42,219,0.4150,0.123288,0.200296
7,Amazon,call,260,2025-09-19,0.330,0.34,219,0.3350,0.219178,0.200619
9,Amazon,call,235,2025-08-15,0.800,0.80,219,0.8000,0.123288,0.166529
10,Amazon,call,250,2025-09-19,0.570,0.58,219,0.5750,0.219178,0.181675


In [ ]:
# étape 3 : simuler le prix des options de marchés avec le modèle de Heston 

r=0.02 # risk free

# paramètres initiaux, non calibrés
sigma = 0.5
kappa = 1
theta= 0.05
volvol= 0.025
rho =-0.5


# Calcl du prix avec le modèle de Heston avec les paramètres non calibré
df["Heston_price"] = df.apply(
    lambda row: hp.call_priceHestonMid(
        row['Spot'],
        row["Prix d'exercice"],
        r,
        row["T"],
        sigma,
        kappa,
        theta,
        volvol,
        rho
    ),
    axis=1
)

df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market,Heston_price
0,Amazon,call,240,2025-07-18,0.070,0.08,219,0.0750,0.046575,0.204805,-0.063176
1,Amazon,call,230,2025-07-18,0.270,0.28,219,0.2750,0.046575,0.158086,0.169928
2,Amazon,call,220,2025-07-18,0.850,0.88,219,0.8650,0.046575,0.064673,2.546705
3,Amazon,call,250,2025-07-18,0.001,0.05,219,0.0255,0.046575,0.244612,-0.129630
4,Amazon,call,255,2025-08-15,0.210,0.22,219,0.2150,0.123288,0.226159,-0.146994
5,Amazon,call,280,2025-09-19,0.110,0.12,219,0.1150,0.219178,0.229892,-0.373131
6,Amazon,call,245,2025-08-15,0.410,0.42,219,0.4150,0.123288,0.200296,-0.022406
7,Amazon,call,260,2025-09-19,0.330,0.34,219,0.3350,0.219178,0.200619,-0.126244
9,Amazon,call,235,2025-08-15,0.800,0.80,219,0.8000,0.123288,0.166529,0.391121
10,Amazon,call,250,2025-09-19,0.570,0.58,219,0.5750,0.219178,0.181675,0.073520


In [ ]:
# On en garde que les valeurs où le prix de Heston est positif

df = df[df['Heston_price'] > 0]
df = df[df['T'] > 2/12] # On retiré les optiosn qui arrivent à expiration
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market,Heston_price
10,Amazon,call,250,2025-09-19,0.57,0.58,219,0.575,0.219178,0.181675,0.073520
12,Amazon,call,240,2025-09-19,0.95,0.96,219,0.955,0.219178,0.156288,0.583804
13,Amazon,call,230,2025-09-19,1.54,1.56,219,1.550,0.219178,0.120746,2.203124
17,Amazon,call,220,2025-09-19,2.32,2.35,219,2.335,0.219178,0.057543,6.277103
18,Amazon,call,290,2026-03-20,0.86,0.89,219,0.875,0.717808,0.192575,0.008534
20,Amazon,call,245,2025-12-19,1.77,1.79,219,1.780,0.468493,0.145055,1.569636
22,Amazon,call,270,2026-03-20,1.45,1.49,219,1.470,0.717808,0.171892,0.667417
23,Amazon,call,225,2025-12-19,3.11,3.15,219,3.130,0.468493,0.080105,7.213580
25,Amazon,call,250,2026-03-20,2.36,2.40,219,2.380,0.717808,0.141361,2.414595
26,Amazon,call,310,2026-09-18,1.42,1.45,219,1.435,1.216438,0.190169,0.381323


In [ ]:
# étape 4 : Obtenir la vol implicte des options calculé avec le modèle de Heston
df['VI_heston'] = df.apply(
    lambda row: iv.call_implied_volatility_mc(
        row['Heston_price'], 
        row['Spot'], 
        row['Prix d\'exercice'], 
        row['T'], 
        r
    ), 
    axis=1
)
df.dropna(inplace=True)
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market,Heston_price,VI_heston
10,Amazon,call,250,2025-09-19,0.57,0.58,219,0.575,0.219178,0.181675,0.073520,0.126065
12,Amazon,call,240,2025-09-19,0.95,0.96,219,0.955,0.219178,0.156288,0.583804,0.136760
13,Amazon,call,230,2025-09-19,1.54,1.56,219,1.550,0.219178,0.120746,2.203124,0.141135
17,Amazon,call,220,2025-09-19,2.32,2.35,219,2.335,0.219178,0.057543,6.277103,0.153945
18,Amazon,call,290,2026-03-20,0.86,0.89,219,0.875,0.717808,0.192575,0.008534,0.104884
20,Amazon,call,245,2025-12-19,1.77,1.79,219,1.780,0.468493,0.145055,1.569636,0.139206
22,Amazon,call,270,2026-03-20,1.45,1.49,219,1.470,0.717808,0.171892,0.667417,0.143157
23,Amazon,call,225,2025-12-19,3.11,3.15,219,3.130,0.468493,0.080105,7.213580,0.149748
25,Amazon,call,250,2026-03-20,2.36,2.40,219,2.380,0.717808,0.141361,2.414595,0.142077
26,Amazon,call,310,2026-09-18,1.42,1.45,219,1.435,1.216438,0.190169,0.381323,0.148363


In [135]:
# étape 5 : pour chacune des options calculer le véga 
from scipy.stats import norm
import numpy as np

def black_scholes_vega(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

df['Vega'] = df.apply(lambda row: black_scholes_vega(row['Spot'], row["Prix d'exercice"], row["T"], r, row['VI_market']), axis=1)
df

,Sous-jacent,Type,Prix d'exercice,Maturité,Bid,Ask,Spot,Mid,T,VI_market,Heston_price,VI_heston,Vega
10,Amazon,call,250,2025-09-19,0.57,0.58,219,0.575,0.219178,0.181675,0.073520,0.126065,14.038312
12,Amazon,call,240,2025-09-19,0.95,0.96,219,0.955,0.219178,0.156288,0.583804,0.136760,20.993809
13,Amazon,call,230,2025-09-19,1.54,1.56,219,1.550,0.219178,0.120746,2.203124,0.141135,30.616370
17,Amazon,call,220,2025-09-19,2.32,2.35,219,2.335,0.219178,0.057543,6.277103,0.153945,40.901748
18,Amazon,call,290,2026-03-20,0.86,0.89,219,0.875,0.717808,0.192575,0.008534,0.104884,22.213909
20,Amazon,call,245,2025-12-19,1.77,1.79,219,1.780,0.468493,0.145055,1.569636,0.139206,36.781287
22,Amazon,call,270,2026-03-20,1.45,1.49,219,1.470,0.717808,0.171892,0.667417,0.143157,33.209124
23,Amazon,call,225,2025-12-19,3.11,3.15,219,3.130,0.468493,0.080105,7.213580,0.149748,57.260180
25,Amazon,call,250,2026-03-20,2.36,2.40,219,2.380,0.717808,0.141361,2.414595,0.142077,48.228309
26,Amazon,call,310,2026-09-18,1.42,1.45,219,1.435,1.216438,0.190169,0.381323,0.148363,34.368514


In [ ]:
# étape  6 : établir la fonction à minimiser (différence entre les vols implicite pondéré par le vega)

def residuals_vega_weighted(params, df, r=0.02):
    kappa, theta, sigma, rho, volvol = params
    res = []

    for i, row in df.iterrows():
        try:
            S = row['Spot']
            K = row["Prix d'exercice"]
            T = row['T']
            market_price = row['Mid']

            # --- Calcul du prix modèle (Heston) ---
            model_price = hp.call_priceHestonMid(S, K, r, T, sigma, kappa, theta, volvol, rho)

            # --- Calcul du vega Black-Scholes ---
            vega = black_scholes_vega(S, K, T, r, row['VI_market'])  

            # Si vega ou prix invalide, on pénalise fortement
            if not np.isfinite(model_price) or not np.isfinite(vega) or vega < 1e-8:
                res.append(1e6)
            else:
                res.append(np.sqrt(vega) * (model_price - market_price)) # On utilise la racine carrée du vega car la fonction least square élève au carré la fonction résidu

        except Exception as e:
            print(f"Erreur ligne {i}: {e}")
            res.append(1e6)

    return np.array(res)


In [137]:
from scipy.optimize import least_squares

#Paramètres initiaux 
init_params = [1.0, 0.04, 0.5, -0.5, 0.04]  # [kappa, theta, sigma, rho, volvol]

# --- Lancement de la calibration ---
print("Lancement de la calibration pondérée par le Vega")

result = least_squares(
    residuals_vega_weighted,
    init_params,
    args=(df,),
    method='lm',          # méthode Levenberg-Marquardt
    verbose=2
)

# --- Résultats ---
kappa, theta, sigma, rho, volvol = result.x
rmse = np.sqrt(np.mean(result.fun**2))

print("\n Calibration terminée.")
print(f"Paramètres calibrés :\n kappa={kappa:.4f}, theta={theta:.4f}, "
      f"sigma={sigma:.4f}, rho={rho:.4f}, volvol={volvol:.4f}")
print(f"RMSE : {rmse:.4f}")


Lancement de la calibration pondérée par le Vega
`ftol` termination condition is satisfied.
Function evaluations 2989, initial cost 3.9774e+04, final cost 8.8929e+01, first-order optimality 3.98e+03.

✅ Calibration terminée.
Paramètres calibrés :
 kappa=0.0000, theta=44.0739, sigma=0.2608, rho=0.9728, volvol=0.0076
RMSE : 2.5666
